# PRDM10

Data derived from [van de Beek I, et al. (2023) PRDM10 directs FLCN expression in a novel disorder overlapping with Birt-Hogg-Dubé syndrome and familial lipomatosis. Hum Mol Genet. 2023;32(7):1223-1235. PMID:36440963](https://pubmed.ncbi.nlm.nih.gov/36440963/)

In [1]:
import pandas as pd
from IPython.display import display, HTML
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
from pyphetools.creation import *
from pyphetools.visualization import IndividualTable, QcVisualizer, HpoaTableBuilder
from pyphetools.validation import *
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.9.34


In [2]:
parser = HpoParser(hpo_json_file="../hp.json")
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
created_by="ORCID:0000-0002-0736-9199"
print(f"HPO version {hpo_version}")

HPO version 2024-01-16


In [3]:
df = pd.read_excel("input/PRDM10_individuals.xlsx")
df.head(2)

,PMID,title,individual_id,Comment,disease_id,disease_label,transcript,allele_1,variant.comment,age_of_onset,sex,HPO,Multiple lipomas,Fibrofolliculoma,Perifollicular fibroma,Pulmonary cyst,Clear cell renal cell carcinoma
0,str,str,str,optional str,str,str,str,HGVS str,optional str,iso8601,M:F:O:U,na,HP:0001012,HP:0030436,HP:0032225,HP:0032445,HP:0006770
1,PMID:36440963,PRDM10 directs FLCN expression in a novel disorder overlapping with Birt-Hogg-Dubé syndrome and familial lipomatosis,IV-1,NaN,OMIM:620459,Birt-Hogg-Dube syndrome 2,NM_020228.3,c.2030G>A,NaN,P33Y,M,na,observed,observed,observed,excluded,excluded


In [4]:
encoder = CaseTemplateEncoder(df=df, hpo_cr=hpo_cr, created_by=created_by)

Created encoders for 17 fields


In [5]:
individuals = encoder.get_individuals()

In [6]:
vmanager = VariantManager(df=df,
                          individual_column_name="individual_id",
                          cohort_name="PRDM10",
                          transcript="NM_020228.3",
                          allele_1_column_name="allele_1")

[INFO] encoding variant "c.2030G>A"
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_020228.3%3Ac.2030G>A/NM_020228.3?content-type=application%2Fjson


In [7]:
vmanager.to_summary()

,status,count,alleles
0,mapped,1,c.2030G>A
1,unmapped,0,


In [8]:
vmanager.add_variants_to_individuals(individuals)
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1, allelic_requirement=AllelicRequirement.MONO_ALLELIC)
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

In [9]:
individuals = cvalidator.get_error_free_individual_list()
table = IndividualTable(individuals)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
IV-1 (MALE; ),Birt-Hogg-Dube syndrome 2 (OMIM:620459),NM_020228.3:c.2030G>A (heterozygous),Multiple lipomas (HP:0001012); Fibrofolliculoma (HP:0030436); Perifollicular fibroma (HP:0032225); excluded: Pulmonary cyst (HP:0032445); excluded: Clear cell renal cell carcinoma (HP:0006770)
III-2 (MALE; ),Birt-Hogg-Dube syndrome 2 (OMIM:620459),NM_020228.3:c.2030G>A (heterozygous),Multiple lipomas (HP:0001012); Fibrofolliculoma (HP:0030436); Perifollicular fibroma (HP:0032225); excluded: Clear cell renal cell carcinoma (HP:0006770)
III-3 (MALE; ),Birt-Hogg-Dube syndrome 2 (OMIM:620459),NM_020228.3:c.2030G>A (heterozygous),Multiple lipomas (HP:0001012); Clear cell renal cell carcinoma (HP:0006770); excluded: Fibrofolliculoma (HP:0030436); excluded: Perifollicular fibroma (HP:0032225)
III-4 (FEMALE; ),Birt-Hogg-Dube syndrome 2 (OMIM:620459),NM_020228.3:c.2030G>A (heterozygous),Multiple lipomas (HP:0001012); Clear cell renal cell carcinoma (HP:0006770); excluded: Fibrofolliculoma (HP:0030436); excluded: Perifollicular fibroma (HP:0032225); excluded: Pulmonary cyst (HP:0032445)
IV-2 (FEMALE; ),Birt-Hogg-Dube syndrome 2 (OMIM:620459),NM_020228.3:c.2030G>A (heterozygous),Multiple lipomas (HP:0001012); Fibrofolliculoma (HP:0030436); Perifollicular fibroma (HP:0032225); excluded: Clear cell renal cell carcinoma (HP:0006770)
IV-3 (MALE; ),Birt-Hogg-Dube syndrome 2 (OMIM:620459),NM_020228.3:c.2030G>A (heterozygous),Multiple lipomas (HP:0001012); Fibrofolliculoma (HP:0030436); Perifollicular fibroma (HP:0032225); Clear cell renal cell carcinoma (HP:0006770); excluded: Pulmonary cyst (HP:0032445)
IV-4 (FEMALE; ),Birt-Hogg-Dube syndrome 2 (OMIM:620459),NM_020228.3:c.2030G>A (heterozygous),Multiple lipomas (HP:0001012); Fibrofolliculoma (HP:0030436); Perifollicular fibroma (HP:0032225); excluded: Clear cell renal cell carcinoma (HP:0006770)


In [10]:
encoder.output_individuals_as_phenopackets(individuals)

We output 7 GA4GH phenopackets to the directory phenopackets


In [11]:
PMID="PMID:36440963"
builder = HpoaTableBuilder.from_individuals(individuals, created_by)
hpoa_creator = builder.young_adult_onset(PMID,1,1).autosomal_dominant(PMID).build()
df = hpoa_creator.get_dataframe()
df

We found a total of 5 unique HPO terms
Extracted disease: Birt-Hogg-Dube syndrome 2 (OMIM:620459)


,#diseaseID,diseaseName,phenotypeID,phenotypeName,onsetID,onsetName,frequency,sex,negation,modifier,description,publication,evidence,biocuration
0,OMIM:620459,Birt-Hogg-Dube syndrome 2,HP:0001012,Multiple lipomas,,,7/7,,,,,PMID:36440963,PCS,ORCID:0000-0002-0736-9199
1,OMIM:620459,Birt-Hogg-Dube syndrome 2,HP:0030436,Fibrofolliculoma,,,5/7,,,,,PMID:36440963,PCS,ORCID:0000-0002-0736-9199
2,OMIM:620459,Birt-Hogg-Dube syndrome 2,HP:0032225,Perifollicular fibroma,,,5/7,,,,,PMID:36440963,PCS,ORCID:0000-0002-0736-9199
3,OMIM:620459,Birt-Hogg-Dube syndrome 2,HP:0032445,Pulmonary cyst,,,0/3,,,,,PMID:36440963,PCS,ORCID:0000-0002-0736-9199
4,OMIM:620459,Birt-Hogg-Dube syndrome 2,HP:0006770,Clear cell renal cell carcinoma,,,3/7,,,,,PMID:36440963,PCS,ORCID:0000-0002-0736-9199
5,OMIM:620459,Birt-Hogg-Dube syndrome 2,HP:0011462,Young adult onset,,,1/1,,,,,PMID:36440963,PCS,ORCID:0000-0002-0736-9199
6,OMIM:620459,Birt-Hogg-Dube syndrome 2,HP:0000006,Autosomal dominant inheritance,,,,,,,,PMID:36440963,PCS,ORCID:0000-0002-0736-9199


In [12]:
hpoa_creator.write_data_frame()

Wrote HPOA disease file to OMIM-620459.tab
